<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/llama_index_to_read_a_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [1]:
%pip install llama-index
%pip install langchain
%pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.8/548.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 

In [17]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


# how to red file in Google Drive

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile().GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title llama_index_to_read_a_book.ipynb, id 1DQDRA1pyRlDNeyTE8gr2Um4h5yWp8BrS
title test-openai.ipynb, id 1wHIqeZB675Kr8QnXEELCkmGdt_T_H3xj
title config.json, id 14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_
title mr_fujino.txt, id 1Z_X79_FrTW69h4SND9BnT9qr3ynsdl5G
title Untitled, id 1I_xhJt_7U59myQaxLAMtlUnz_xmDMSa8
title semantic_search.ipynb, id 1MXkSYu3eavOHrr4FwfQ-FoBVs3LE3bQB
title edit_moderation_text.ipynb, id 1PTfZGQOUWWMDQ41Xl9KGUxI270EMGunn
title text-clusters.ipynb, id 1bYXHDDARDf-pJiSWAttiArQGk_VxE1s0
title Fasttext-lab.ipynb, id 17YcjW4HEBGEToNu1ZAqMT-iOrKeMf-Ax
title T5.ipynb, id 1LLV0UCHCmXJ16diU5-jmkh47QysRB6SW
title Colab Notebooks, id 1hjY5nzNXXO7Q9xn449qgGaVZmnW8aCic
title React Server Components, id 1a4yskaYi7Ons0LphfyBbFd3t32c39rJDOxv0WwAALBM
title rust学习笔记, id 1XP8BLuZmnKMMfiuuN-lXqF8mm8J3R_cHz-pP-T0q36g
title ChatGpt相关资料, id 1ky1sQiAZcJPk9FduOzoMrrok7JzHox3SGCLKXn0da3k
title 提案, id 18CHXMKdoSeuP5q2eTS6u5ECPQcdhzF2wiDQxte7PBTY
title 2023, id 1nURL1_VBTpo48UX1tV24PXCvvgun3_7Eo

# 先搜索，后提示


In [2]:
from google.colab import drive, files
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
import io
import json
import openai
from llama_index import Document, GPTSimpleVectorIndex
import os

auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

## 藤野先生txt文档
data_file_id = "1Z_X79_FrTW69h4SND9BnT9qr3ynsdl5G"
data_file = drive_service.files().get(fileId=data_file_id).execute()
content = drive_service.files().get_media(fileId=data_file_id).execute().decode()
# print(content)

doc = Document(content)

# print(doc)
# # create an index from the document object
index = GPTSimpleVectorIndex.from_documents([doc])


# # save the index to disk
response = index.query("鲁迅先生在日本学习医学的老师是谁？")
print(response)





鲁迅先生在日本学习医学的老师是藤野严九郎，他曾经问过鲁迅关于中国女性裹脚的情况，并且有一次他的学生会干事到鲁迅寓里来借讲义，之后鲁迅收到了一封来自藤野严九郎的信。


In [3]:
response = index.query("鲁迅先生是去哪里学的医学？", verbose=True)
print(response)


鲁迅先生去仙台学习医学。


In [4]:
from llama_index import QuestionAnswerPrompt
query_str = "鲁迅先生去哪里学的医学？"
DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)
QA_PROMPT = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL)

response = index.query(query_str, text_qa_template=QA_PROMPT, verbose=True)
print(response)


鲁迅先生去日本的仙台学习医学。


In [5]:
QA_PROMPT_TMPL = (
    "下面的“我”指的是鲁迅先生 \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "根据这些信息，请回答问题: {query_str}\n"
    "如果您不知道的话，请回答不知道\n"
)
QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)

response = index.query("请问林黛玉和贾宝玉是什么关系？", text_qa_template=QA_PROMPT)

print(response)


不知道


# 通过llama_index对于文章进行小结


In [12]:
%pip install spacy
!python -m spacy download zh_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-25 13:14:11.783916: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 13:14:13.050061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')


In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import SpacyTextSplitter
from llama_index import GPTListIndex, LLMPredictor, ServiceContext
from llama_index.node_parser import SimpleNodeParser

# define LLM
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=1024))

text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
parser = SimpleNodeParser(text_splitter=text_splitter)
## 藤野先生txt文档
data_file_id = "1Z_X79_FrTW69h4SND9BnT9qr3ynsdl5G"
data_file = drive_service.files().get(fileId=data_file_id).execute()
content = drive_service.files().get_media(fileId=data_file_id).execute().decode()
# print(content)

doc = Document(content)
nodes = parser.get_nodes_from_documents([doc])

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

list_index = GPTListIndex(nodes=nodes, service_context=service_context)
response = list_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", response_mode="tree_summarize")
print(response)

鲁迅先生在日本留学期间，买书时会听到学生跳舞的声音。他后来去了仙台的医学专门学校，那里没有中国的学生，但他得到了免费的食宿和职员的照顾。他在那里学习解剖学，遇到了一些困难和歧视，但他仍然努力学习，取得了不错的成绩。最后，他决定不再学医，而是转学生物学。他回忆起他的解剖学老师，感慨他的热心希望中国有新的医学，同时也为学术的发展而努力。他收藏了老师的讲义，但在搬家时遗失了一部分。他仍然常常想起老师，他的形象和教诲一直鼓励着他写作。


In [16]:
from llama_index import GPTTreeIndex

# define LLM
tree_index = GPTTreeIndex(nodes=nodes, service_context=service_context)
response = tree_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", mode="summarize")
print(response)

鲁迅先生描述了他在日本留学期间的生活，包括在中国留学生会馆买书、在医学专门学校学习、遇到的困难和歧视、取得的成绩以及最终决定转学生物学。他回忆起自己的解剖学老师，感慨老师的热心和为学术发展而努力，收藏了老师的讲义，但不幸遗失了一部分。老师的形象和教诲一直鼓励着他写作。


# 支持图片的多模态


In [30]:
from llama_index import download_loader, GPTSimpleVectorIndex
from llama_index.readers.file.base import DEFAULT_FILE_EXTRACTOR, ImageParser
from llama_index.response.notebook_utils import display_response, display_image
from llama_index.indices.query.query_transform.base import ImageOutputQueryTransform

image_parser = ImageParser(keep_image=True, parse_text=True)
file_extractor = DEFAULT_FILE_EXTRACTOR
file_extractor.update(
{
    ".jpg": image_parser,
    ".png": image_parser,
    ".jpeg": image_parser,
})

# NOTE: we add filename as metadata for all documents
filename_fn = lambda filename: {'file_name': filename}

# RemoteDepthReader = download_loader("RemoteDepthReader")

# loader = RemoteDepthReader(
#   file_extractor=file_extractor
# )
# receipt_documents = loader.load_data(url="https://raw.githubusercontent.com/xuwenhao/geektime-ai-course/1168ef06f4c2b630b676e442839369aae30ac0f2//data/receipts/1100-receipt.jpg",)



RemoteReader = download_loader("RemoteReader")

loader = RemoteReader(
    file_extractor=file_extractor,
)
documents = loader.load_data(url="https://en.wikipedia.org/wiki/File:Example.jpg")


TypeError: ignored

In [23]:
receipts_index = GPTSimpleVectorIndex.from_documents(receipt_documents)

In [24]:
receipts_response = receipts_index.query(
    'When was the last time I went to McDonald\'s and how much did I spend. \
    Also show me the receipt from my visit.',
    query_transform=ImageOutputQueryTransform(width=400)
)

display_response(receipts_response)

**`Final Response:`** None

{}